In [ ]:
# INSTALAÇÃO DE BIBLIOTECAS
from IPython.display import clear_output
!pip install scikeras
!pip install scikit-learn==1.4.2
!pip install tensorflow
!pip install tabulate
!pip install seaborn
!pip install xgboost
!pip install imblearn
!pip install shap
clear_output()

In [ ]:
# BIBLIOTECAS
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import xgboost as XGB
import shap

from IPython.display import clear_output
from IPython.display import HTML
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from scikeras.wrappers import KerasClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from itertools import product

In [ ]:
# LEITURA DO DATASET
data = pd.read_csv('Maternal Health Risk Data Set.csv')
palette = {'low risk': 'yellowgreen', 'mid risk': 'gold', 'high risk': 'firebrick'}
paleta = ['yellowgreen', 'gold', 'firebrick']

In [ ]:
# FUNÇÃO PARA A DIVISÃO DE DADOS DO DATASET 1
def func_data_1(random_state = 42, output = True):
  # LEITURA DO BANCO DE DADOS
  data_1 = copy.deepcopy(data)

  # DIVISÃO DOS DADOS (FEATURES E TARGET)
  X = data_1[data_1.columns[:6]]
  y = data_1[data_1.columns[6]]

  # TRANSFORMANDO AS CLASSES EM VALORES NUMÉRICOS
  y = y.map({'low risk': 0, 'mid risk': 1, 'high risk': 2})

  # DIVISÃO DOS CONJUNTOS DE TREINO E DE TESTE
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state, stratify = y) # 80% para treino (X_train, y_train) e 20% para teste (X_test, y_test) de forma estratificada
  rs = random_state # garante reprodutibilidade salvando o valor randomico

  # EXIBIÇÃO DE INFORMAÇÕES SOBRE O DATASET
  print('== DATASET ORGINAL ==')
  print('- Classes -\n')
  print('Baixo Risco (0):', len(data_1[data_1['RiskLevel'] == 'low risk']))
  print('Médio Risco (1):', len(data_1[data_1['RiskLevel'] == 'mid risk']))
  print('Alto Risco  (2):', len(data_1[data_1['RiskLevel'] == 'high risk']))
  print(f'\nDistribuição de targets de TREINO: {Counter(y_train)}')
  print(f'Distribuição de targets de TESTE: {Counter(y_test)}')
  print('\n- Informações Adicionais -\n')
  data_1.info()
  print('\n- Tabela -\n')
  display(data_1.head())

  if output == False:
    clear_output()

  return data_1, np.array(X_train), np.array(X_test), y_train, y_test

In [ ]:
data_1, X_train, X_test, y_train, y_test = func_data_1()

In [ ]:
# ESTATÍSTICAS DESCRITIVAS DO DATASET ORIGINAL
# Vetores para armazenar cada registro por classe
lowR = data_1[data_1['RiskLevel'] == 'low risk']
mediumR = data_1[data_1['RiskLevel'] == 'mid risk']
highR = data_1[data_1['RiskLevel'] == 'high risk']
# Exclui linhas vazias
lowR = lowR.reset_index(drop = True)
mediumR = mediumR.reset_index(drop = True)
highR = highR.reset_index(drop = True)

low_discribe = pd.DataFrame(lowR.describe()).round(2)
medium_discribe = pd.DataFrame(mediumR.describe()).round(2)
high_discribe = pd.DataFrame(highR.describe()).round(2)

discribe_all = pd.concat([low_discribe, medium_discribe, high_discribe], keys = ['Baixo Risco', 'Médio Risco', 'Alto Risco'])

display(discribe_all)
discribe_all.to_csv('Discribe_low_mid_high-Dataset_Original.csv')

In [ ]:
# PROPORÇÃO DE DIAGNÓSTICOS
plt.figure(figsize=(8, 5))
plt.bar('Baixo', len(lowR), color =  paleta[0])
plt.bar('Médio', len(mediumR), color =  paleta[1])
plt.bar('Alto', len(highR), color =  paleta[2])
plt.title('Quantidade por Nível de Risco do Dataset Original', fontsize=14)
plt.xlabel('Nível de Rico')
plt.ylabel('Quantidade')
plt.tight_layout()
plt.show()

In [ ]:
# MATRIZ DE CORRELAÇÃO DOS DADOS
# Calcula a matriz de correlação
dataCORR = data_1.drop(['RiskLevel'], axis = 1)
correlation = dataCORR.corr(numeric_only=True)

# Cria o heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(correlation, annot=True, fmt='0.2f',cmap='Blues')
plt.title("Matriz de Correlação", fontsize=14)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# HISTOGRAMAS E BOXPLOTS
for i in range(0, len(data_1.columns) - 1 ):
  fig, ax = plt.subplots(ncols = 2, nrows = 1, figsize = (12, 6))
  ax[0].hist([lowR[data_1.columns[i]], mediumR[data_1.columns[i]], highR[data_1.columns[i]]], bins = 12, color = paleta, alpha = 0.7, label=['Baixo Risco', 'Médio Risco', 'Alto Risco'], histtype = 'bar')
  ax[0].set_title(f'Distribuição de {data_1.columns[i]}', fontsize=14)
  ax[0].set_xlabel(data_1.columns[i])
  ax[0].set_ylabel('Frequência')
  ax[0].legend()
  sns.boxplot(data = data_1, x = 'RiskLevel', y = data_1.columns[i], ax = ax[1], palette = palette, hue='RiskLevel', order = ['low risk', 'mid risk', 'high risk'])
  ax[1].set_title(f'{data_1.columns[i]} por Nível de Risco', fontsize=14)
  plt.tight_layout()
  plt.show()

In [ ]:
# GRÁFICO DE DISPERSÃO
for i in range(0, len(data_1.columns) - 1):
  count = 0
  fig, ax = plt.subplots(ncols = 5, nrows = 1, figsize = (30, 6))
  for j in range(0, len(data_1.columns) - 1):
    if i != j:
      ax[count].scatter(lowR[data_1.columns[i]], lowR[data_1.columns[j]], color = palette['low risk'], label = 'Baixo risco')
      ax[count].scatter(mediumR[data_1.columns[i]], mediumR[data_1.columns[j]], color = palette['mid risk'], label = 'Médio risco')
      ax[count].scatter(highR[data_1.columns[i]], highR[data_1.columns[j]], color = palette['high risk'], label = 'Alto risco')
      ax[count].set_xlabel(data_1.columns[i])
      ax[count].set_ylabel(data_1.columns[j])
      ax[count].set_title(f'{data_1.columns[i]} x {data_1.columns[j]}', fontsize=14)
      ax[count].legend()
      count += 1

  plt.tight_layout()
  plt.show()

In [ ]:
# FUNÇÃO PARA A DIVISÃO DE DADOS DO DATASET 3
def func_data_3(random_state = 42, output = True):
  # LEITURA DO BANCO DE DADOS
  data_3 = copy.deepcopy(data)

  # REFINAMENTO
  for i in range(0, len(data['Age'])):
  # Correção do registro errado da frequência cardíaca
    if data_3['HeartRate'][i] < 20:
      data_3.loc[i, 'HeartRate'] = data_3['HeartRate'][i] * 10
  # Remoção dos registros das gestantes menores de 18
    # elif data_R['Age'][i] < 18:
    #   data_R = data_R.drop(i, axis = 0)
  data_3 = data_3.drop_duplicates().reset_index(drop = True) # Remove registros duplicados e exclui linhas vazias

  # DIVISÃO DOS DADOS (FEATURES E TARGET)
  X = data_3[data_3.columns[:6]]
  y = data_3[data_3.columns[6]]

  # TRANSFORMANDO AS CLASSES EM VALORES NUMÉRICOS
  y = y.map({'low risk': 0, 'mid risk': 1, 'high risk': 2})

  # DIVISÃO DOS CONJUNTOS DE TREINO E DE TESTE
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state, stratify = y) # 80% para treino (X_train, y_train) e 20% para teste (X_test, y_test) de forma estratificada
  rs = random_state # garante reprodutibilidade salvando o valor randomico

  # ALGORITMO SMOTE
  sm = SMOTE(random_state = random_state, k_neighbors = 5) # Configuração do SMOTE
  X_train, y_train = sm.fit_resample(X_train, y_train) # Adição do SMOTE para o set de treinamento

  # DATASET COM REGISTROS SINTÉTICOS
  X_concat = pd.concat([X_train, X_test], axis = 0)
  y_concat = pd.concat([y_train, y_test], axis = 0)
  data_3 = pd.concat([X_concat, y_concat], axis = 1)
  data_3 = data_3.replace({0: 'low risk', 1: 'mid risk', 2: 'high risk'})

  # NORMALIZAÇÃO DOS DADOS
  scaler = preprocessing.MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # EXIBIÇÃO DE INFORMAÇÕES SOBRE O DATASET
  print('== DATASET REFINADO ==')
  print('- Classes -\n')
  print('Baixo Risco (0):', len(data_3[data_3['RiskLevel'] == 'low risk']))
  print('Médio Risco (1):', len(data_3[data_3['RiskLevel'] == 'mid risk']))
  print('Alto Risco  (2):', len(data_3[data_3['RiskLevel'] == 'high risk']))
  print(f'\nDistribuição de targets de TREINO: {Counter(y_train)}')
  print(f'Distribuição de targets de TESTE: {Counter(y_test)}')
  print('\n- Informações Adicionais -\n')
  data_3.info()
  print('\n- Tabela -\n')
  display(data_3.head())

  if output == False:
    clear_output()

  return data_3, X_train, X_test, y_train, y_test

In [ ]:
# CHAMADA DA FUNÇÃO
data_3, X_train, X_test, y_train, y_test = func_data_3()

# DATASET NORMALIZADO
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)
X_concat = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_test)], axis = 0)
y_concat = pd.concat([y_train, y_test], axis = 0)
data_3_norm = pd.concat([X_concat, y_concat], axis = 1)
data_3_norm.columns = data_3.columns

In [ ]:
# ESTATÍSTICAS DESCRITIVAS DO DATASET ORIGINAL
# Vetores para armazenar as estatísticas descritivas
lowR = data_3_norm[data_3_norm['RiskLevel'] == 0]
mediumR = data_3_norm[data_3_norm['RiskLevel'] == 1]
highR = data_3_norm[data_3_norm['RiskLevel'] == 2]

# Exclui linhas vazias
lowR = lowR.reset_index(drop = True)
mediumR = mediumR.reset_index(drop = True)
highR = highR.reset_index(drop = True)

low_discribe = pd.DataFrame(lowR.describe()).round(4)
medium_discribe = pd.DataFrame(mediumR.describe()).round(4)
high_discribe = pd.DataFrame(highR.describe()).round(4)

discribe_all = pd.concat([low_discribe, medium_discribe, high_discribe], keys = ['Baixo Risco', 'Médio Risco', 'Alto Risco'])

display(discribe_all)
discribe_all.to_csv('Discribe_low_mid_high-Dataset_Refinado+SMOTE.csv')

In [ ]:
# PROPORÇÃO DE DIAGNÓSTICOS
plt.figure(figsize=(8, 5))
plt.bar('Baixo', len(lowR), color = paleta[0])
plt.bar('Médio', len(mediumR), color = paleta[1])
plt.bar('Alto', len(highR), color = paleta[2])
plt.title('Quantidade por Nível de Risco do Dataset Original', fontsize=14)
plt.xlabel('Nível de Rico')
plt.ylabel('Quantidade')
plt.tight_layout()
plt.show()

In [ ]:
# MATRIZ DE CORRELAÇÃO DOS DADOS
# Calcula a matriz de correlação
dataCORR = data_3.drop(['RiskLevel'], axis = 1)
correlation = dataCORR.corr(numeric_only=True)

# Cria o heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(correlation, annot=True, fmt='0.2f',cmap='Blues')
plt.title("Matriz de Correlação", fontsize=14)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# HISTOGRAMAS E BOXPLOTS
data_3_norm['RiskLevel'] = data_3_norm['RiskLevel'].map({0: 'low risk', 1: 'mid risk', 2: 'high risk'})
for i in range(0, len(data_3.columns) - 1 ):
  fig, ax = plt.subplots(ncols = 2, nrows = 1, figsize = (12, 6))
  ax[0].hist([lowR[data_3.columns[i]], mediumR[data_3.columns[i]], highR[data_3.columns[i]]], bins = 12, color = paleta, alpha = 0.7, label=['Baixo Risco', 'Médio Risco', 'Alto Risco'], histtype = 'bar')
  ax[0].set_title(f'Distribuição de {data_3.columns[i]}', fontsize=14)
  ax[0].set_xlabel(data_3.columns[i])
  ax[0].set_ylabel('Frequência')
  ax[0].legend()
  sns.boxplot(data = data_3_norm, x = 'RiskLevel', y = data_3.columns[i], ax = ax[1], palette = palette, hue='RiskLevel', order = ['low risk', 'mid risk', 'high risk'])
  ax[1].set_title(f'{data_3.columns[i]} por Nível de Risco', fontsize=14)
  plt.tight_layout()
  plt.show()

In [ ]:
# Vetores para armazenar cada registro normalizado por classe
# GRÁFICO DE DISPERSÃO
for i in range(0, len(data_3.columns) - 1):
  count = 0
  fig, ax = plt.subplots(ncols = 5, nrows = 1, figsize = (30, 6))
  for j in range(0, len(data_3.columns) - 1):
    if i != j:
      ax[count].scatter(lowR[data_3.columns[i]], lowR[data_3.columns[j]], color = palette['low risk'], label = 'Baixo risco')
      ax[count].scatter(mediumR[data_3.columns[i]], mediumR[data_3.columns[j]], color = palette['mid risk'], label = 'Médio risco')
      ax[count].scatter(highR[data_3.columns[i]], highR[data_3.columns[j]], color = palette['high risk'], label = 'Alto risco')
      ax[count].set_xlabel(data_3.columns[i])
      ax[count].set_ylabel(data_3.columns[j])
      ax[count].set_title(f'{data_3.columns[i]} x {data_3.columns[j]}', fontsize=14)
      ax[count].legend()
      count += 1

  plt.tight_layout()
  plt.show()

# **MLP**

## Método Hold-Out #1

In [ ]:
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
data_, X_train, X_test, y_train, y_test = func_data_3(output = False) # !!! escolha o dataset alterando o número após o func_data_ !!!
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

# TREINO E VALIDAÇÃO HOLD-OUT
# Definição de uma função para a compilação de um modelo variável
def build_model(hidden1, hidden2, hidden3, dropout1, dropout2, dropout3, activation, learning_rate):
  model = Sequential([
      Input(shape = (6,)),
      Dense(hidden1, activation = activation),
      Dropout(dropout1),
      Dense(hidden2, activation = activation),
      Dropout(dropout2),
      Dense(hidden3, activation = activation),
      Dropout(dropout3),
      Dense(3, activation = 'softmax')
      ])
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics = ['accuracy'])
  return model

# Hiperparâmetros para teste
hidden1_options = [256]
hidden2_options = [128]
hidden3_options = [64]
dropout1_options = [0.5]
dropout2_options = [0.5]
dropout3_options = [0.0]
activation_options = ['leaky_relu']
learning_rate_options = [0.001]
batch_size_options = [32]
# hidden1_options = [128, 256, 512]
# hidden2_options = [64, 128, 256]
# hidden3_options = [32, 64, 128]
# dropout1_options = [0.0, 0.3, 0.5]
# dropout2_options = [0.0, 0.3, 0.5]
# dropout3_options = [0.0, 0.3, 0.5]
# activation_options = ['relu', 'leaky_relu']
# learning_rate_options = [0.01, 0.001]
# batch_size_options = [16, 32, 48]

tot = len(hidden1_options) * len(hidden2_options) * len(hidden3_options) * len(dropout1_options) * len(dropout2_options) * len(dropout3_options) * len(activation_options) * len(learning_rate_options) * len(batch_size_options)

# Lista para armazenar os resultados
results = []

# Definição do Eraly Stopping
callback_HoldOut_GS = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20, restore_best_weights = True) # Ferramenta de early stopping

count = 0
# EXECUÇÃO DO GRID SEARCH
for h1, h2, h3, dp1, dp2, dp3, act, lr, bs in product(hidden1_options, hidden2_options, hidden3_options, dropout1_options, dropout2_options, dropout3_options, activation_options, learning_rate_options, batch_size_options):
  count += 1
  print( f'Testando: hidden1 = {h1}, hidden2 = {h2}, hidden2 = {h3}, dropout1 = {dp1}, dropout1 = {dp2}, dropout1 = {dp3}, activation = {act}, learning rate = {lr}, batch_size = {bs}')
  model = build_model(h1, h2, h3, dp1, dp2, dp3, act, lr)
  model.fit(X_train, y_train, epochs = 300, batch_size = bs, callbacks = callback_HoldOut_GS, verbose = 0, validation_data = (X_test, y_test), shuffle = True) # Treina o modelo e avalia o desempenho com os dados de teste em cada época
  y_pred = np.argmax(model.predict(X_test), axis=1)
  acc = accuracy_score(y_test, y_pred)
  results.append(((h1, h2, h3, dp1, dp2, dp3, act, lr, bs), acc))
  print(f'{count}/{tot}')

In [ ]:
# OBTER O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
best_model = max(results, key=lambda x: x[1])
h1, h2, h3, dp1, dp2, dp3, act, lr, bs = best_model[0]
print(f'Melhor modelo: hidden1 = {h1}, hidden2 = {h2}, hidden2 = {h3}, dropout1 = {dp1}, dropout2 = {dp2}, dropout3 = {dp3}, activation = {act}, learning rate = {lr}, batch_size = {bs}')

In [ ]:
# AVALIAR 10x O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
final_model = build_model(h1, h2, h3, dp1, dp2, dp3, act, lr)

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1
  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  data_, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  history = final_model.fit(X_train, y_train, epochs = 500, batch_size = bs, callbacks = callback_HoldOut_GS, verbose = 0, validation_data = (X_test, y_test), shuffle = True)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = np.argmax(final_model.predict(X_test), axis=1)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict(X_test) # Previsões
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')

## Validação K-Fold #2

In [ ]:
# TREINO E VALIDAÇÃO CRUZADA K-FOLD
# Hiperparâmetros para teste (os melhores do grid hold-out)
param_grid = {
    'model__hidden1': [h1],
    'model__hidden2': [h2],
    'model__hidden3': [h3],
    'model__dropout1': [dp1],
    'model__dropout2': [dp2],
    'model__dropout3': [dp3],
    'model__activation': [act],
    'model__learning_rate': [lr],
    'batch_size': [bs]
}

# Definição do Eraly Stopping
callback_KFold = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 20, restore_best_weights = True)

# Criar classificador
clf = KerasClassifier(
    model = build_model,
    callbacks = callback_KFold,
    epochs = 300,
    verbose = 0
)

# Lista para armazenar os resultados
results = []

# Aplicar GridSearchCV para K = 2, 3, ..., 5
for i in range(1, 5):
  K = i+1
  print(f'Validação K-Fold com {K} folds em andamento...')
  # Treinando o modelo no K-Fold
  grid = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)
  y_pred = grid.predict(X_test)
  results.append(accuracy_score(y_test, y_pred))
  print('Concluida.')

In [ ]:
# Exibindo todas as acurácias
for i in range(1, 5):
  print(f'{i+1}-Fold, acurácia:', results[i-1])
# Exibindo o melhor K e sua acurácia
print(f'Melhor K = {results.index(max(results)) + 2} com acurácia = {max(results)}')

In [ ]:
# AVALIAR 10x O MELHOR K DA VALIDAÇÃO K-FOLD
K = results.index(max(results)) + 2

# Definição do Eraly Stopping
callback_KFold = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 20, restore_best_weights = True)

# Criar classificador
clf = KerasClassifier(
    model = build_model,
    callbacks = callback_KFold,
    epochs = 300,
    verbose = 0
)

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1

  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  data_, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  # Treinando o modelo
  grid = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = grid.predict(X_test)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict(X_test) # Previsões
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')

## Gráficos SHAP

In [ ]:
explainer = shap.Explainer(final_model, X_train)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test, feature_names = data_.columns[:6])
shap.plots.bar(shap.Explanation(values = shap_values.values, base_values = shap_values.base_values, data = X_test, feature_names = data_.columns[:6]), max_display = 6)

# Inicializa o visualizador JS do SHAP
shap.initjs()

# Seleciona a instancia a ser explicada (exemplo: primeira amostra do teste)
i = 0 # ou qualquer outro índice

# Criação do gráfico
force_plot_html = shap.plots.force(
    base_values = shap_values.base_values[i],
    shap_values = shap_values.values[i],
    features = X_test[i],
    feature_names = data_.columns[:6],
    matplotlib = False,
    show = False
).html()

# Exibir o gráfico corretamente no  Colab
display(HTML(force_plot_html))"

# **Random Forest**

## Hold-Out #1

In [ ]:
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
_, X_train, X_test, y_train, y_test = func_data_3(output = False) # !!! escolha o dataset alterando o número após o func_data_ !!!
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

# TREINO E VALIDAÇÃO HOLD-OUT
# Hiperparâmetros para teste
n_estimators_options = [100, 200, 300, 500] # Número de árvores na floresta
max_depth_options = [None, 10, 20, 30] # Profundidade máxima da árvore (None significa nós expandidos até que todas as folhas sejam puras ou contenham menos min_samples_split amostras)
min_samples_split_options = [2, 5, 10] # Número mínimo de amostras necessárias para dividir um nó interno
min_samples_leaf_options = [1, 2, 4] # Número mínimo de amostras necessárias para estar em um nó folha
max_features_options = ['sqrt', 'log2', 1.0] # Número de recursos a serem considerados ao procurar a melhor divisão ('sqrt' é geralmente padrão e bom)
bootstrap_options = [True, False] # Se amostras de bootstrap são usadas ao construir árvores

# Lista para armazenar os resultados
results = []

# EXECUÇÃO DO GRID SEARCH
for n_est, m_depth, min_split, min_leaf, max_feat, bootstrap in product(n_estimators_options, max_depth_options, min_samples_split_options, min_samples_leaf_options, max_features_options, bootstrap_options):
    print(f'Testando: n_estimators = {n_est}, max_depth = {m_depth}, min_samples_split = {min_split}, min_samples_leaf = {min_leaf}, max_features = {max_feat}, bootstrap = {bootstrap}')
    # Instanciando o modelo com os hiperparâmetros atuais
    model = RandomForestClassifier(n_estimators = n_est, max_depth = m_depth, min_samples_split = min_split, min_samples_leaf = min_leaf, max_features = max_feat, bootstrap = bootstrap)
    # Treina o modelo
    model.fit(X_train, y_train)

    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Calcula a acurácia
    acc = accuracy_score(y_test, y_pred)
    results.append(((n_est, m_depth, min_split, min_leaf, max_feat, bootstrap), acc))

In [ ]:
# OBTER O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
best_model = max(results, key=lambda x: x[1])
n_est, m_depth, min_split, min_leaf, max_feat, bootstrap = best_model[0]
print(f'Melhor modelo: n_estimators = {n_est}, max_depth = {m_depth}, min_samples_split = {min_split}, min_samples_leaf = {min_leaf}, max_features = {max_feat}, bootstrap = {bootstrap}')

In [ ]:
# AVALIAR 10x O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
final_model = RandomForestClassifier(n_estimators = n_est, max_depth = m_depth, min_samples_split = min_split, min_samples_leaf = min_leaf, max_features = max_feat, bootstrap = bootstrap)

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1
  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  _, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  history = final_model.fit(X_train, y_train)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = model.predict(X_test)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict_proba(X_test) # Previsões de probabilidade
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')

## Validação K-Fold #2

In [ ]:
# TREINO E VALIDAÇÃO CRUZADA K-FOLD
# Hiperparâmetros para teste (os melhores do grid hold-out)
param_grid = {
    'n_estimators': [n_est],
    'max_depth': [m_depth],
    'min_samples_split': [min_split],
    'min_samples_leaf': [min_leaf],
    'max_features': [max_feat],
    'bootstrap': [bootstrap]
    }

# Lista para armazenar os resultados
results = []

# Aplicar GridSearchCV para K = 2, 3, ..., 5
for i in range(1, 5):
  K = i+1
  print(f'Validação K-Fold com {K} folds em andamento...')
  grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)
  y_pred = grid.predict(X_test)
  results.append(accuracy_score(y_test, y_pred))
  print('Concluida.')

In [ ]:
# Exibindo todas as acurácias
for i in range(1, 5):
  print(f'{i+1}-Fold, acurácia:', results[i-1])
# Exibindo o melhor K e sua acurácia
print(f'Melhor K = {results.index(max(results)) + 2} com acurácia = {max(results)}')

In [ ]:
# AVALIAR 10x O MELHOR K DA VALIDAÇÃO K-FOLD
K = results.index(max(results)) + 2

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1

  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  _, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  # Treinando o modelo
  grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = grid.predict(X_test)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict_proba(X_test) # Previsões
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')

Gráficos Shap:

In [ ]:
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import xgboost as XGB
import shap

from IPython.display import clear_output
from IPython.display import HTML
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, roc_auc_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from scikeras.wrappers import KerasClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from itertools import product

In [ ]:
# EXIBIÇÃO DOS GRÁFICOS SHAP
class_names = ['baixo risco', 'médio risco', 'alto risco']
explainer = shap.TreeExplainer(final_model)
shap_values = explainer(X_test)

# Gráfico resumo geral
shap.summary_plot(shap_values, X_test, feature_names = np.array(data.columns[:6]), plot_type = 'bar', show = False, class_names = class_names, color = plt.get_cmap("tab20c"))
plt.title("Gráfico Resumo das Classes")
plt.show()
print('=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x')

# Gráficos de resumo, barra e de força de cada classe
instance_index = 0 # Seleciona a instancia a ser explicada (exemplo: primeira linha do teste (X_test[0]))
for i in range(0, len(class_names)):
  shap.summary_plot(shap_values[:, :, i], X_test, feature_names = np.array(data.columns[:6]), show = False) # quanto mais para a direita mais a influencia para a classe i
  plt.title(f'Gráfico Resumo da classe {class_names[i]}')
  plt.show()

  shap.plots.bar(shap.Explanation(values = shap_values[:, :, i], base_values = shap_values.base_values, data = X_test, feature_names = np.array(data.columns[:6])), max_display = 6, show = False)
  plt.title(f'Gráfico de Barra da classe {class_names[i]}')
  plt.show()

  # Escolhe a classe que você quer explicar (por exemplo, a classe 0)
  class_to_explain = i

  # SHAP values para a instância e classe escolhida
  shap_values_instance_class = shap_values[instance_index, :, class_to_explain]

  # Base value (expected_value) para a classe escolhida
  base_value_for_class = shap_values.base_values[class_to_explain]

  # Valores das features para a instância escolhida
  feature_values_instance = X_test[instance_index]

  explanation_for_instance = shap.Explanation(
      values = shap_values_instance_class,
      base_values = base_value_for_class,
      data = feature_values_instance,
      feature_names = np.array(data.columns[:6])
  )
  shap.plots.force(explanation_for_instance, matplotlib = True, show = False)
  plt.title(f'Gráfico de Força da classe {class_names[i]}')
  plt.show()
  print('=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x=x')

# **SVM**

## Método Hold-Out #1

In [ ]:
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
_, X_train, X_test, y_train, y_test = func_data_3(output = False) # !!! escolha o dataset alterando o número após o func_data_ !!!
''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

# TREINO E VALIDAÇÃO HOLD-OUT
# Hiperparâmetros para teste
C_options = [0.1, 1, 10, 100] # Parâmetro de regularização
kernel_options = ['linear', 'poly', 'rbf', 'sigmoid'] # Tipo de kernel
gamma_options = ['scale', 'auto'] # Coeficiente do kernel (para 'rbf', 'poly' e 'sigmoid')
degree_options = [2, 3, 4] # Grau do polinômio para o kernel 'poly'

# Lista para armazenar os resultados
results = []

# EXECUÇÃO DO GRID SEARCH
for C, kernel, gamma, degree in product(C_options, kernel_options, gamma_options, degree_options):
    # Ajuste para evitar combinações inválidas (degree só se aplica a kernel='poly')
    if kernel != 'poly' and degree != degree_options[0]: # Usamos o primeiro valor como padrão quando não é poly
        continue

    print(f'Testando: C = {C}, kernel = {kernel}, gamma = {gamma}, degree = {degree}')

    # Criação do modelo SVM com os hiperparâmetros atuais
    if kernel == 'poly':
        model = SVC(C = C, kernel = kernel, gamma = gamma, degree = degree, probability = True)
    else:
        model = SVC(C = C, kernel = kernel, gamma = gamma, probability = True)

    # Treina o modelo com os dados de treino
    history = model.fit(X_train, y_train)

    # Faz previsões nos dados de teste
    y_pred = model.predict(X_test)

    # Avalia a acurácia
    acc = accuracy_score(y_test, y_pred)
    results.append(((C, kernel, gamma, degree), acc))

In [ ]:
# OBTER O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
best_model = max(results, key=lambda x: x[1])
C, kernel, gamma, degree = best_model[0]
if kernel == 'poly':
  print(f'Melhor modelo: C = {C}, kernel = {kernel}, gamma = {gamma}, degree = {degree}')
else:
  print(f'Melhor modelo: C = {C}, kernel = {kernel}, gamma = {gamma}')

In [ ]:
# AVALIAR 10x O MELHOR MODELO DA VALIDAÇÃO HOLD-OUT
if kernel == 'poly':
  final_model = SVC(C = C, kernel = kernel, gamma = gamma, degree = degree, probability = True)
else:
  final_model = SVC(C = C, kernel = kernel, gamma = gamma, probability = True)

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1
  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  _, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  history = final_model.fit(X_train, y_train)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = model.predict(X_test)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict_proba(X_test) # Previsões
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')

## Validação K-Fold #2

In [ ]:
# TREINO E VALIDAÇÃO CRUZADA K-FOLD
# Hiperparâmetros para teste (os melhores do grid hold-out)
if kernel == 'poly':
  param_grid = {
        'kernel': ['poly'],
        'C': [C],
        'gamma': [gamma],
        'degree': [degree]
    }
else:
  param_grid = {
        'kernel': [kernel],
        'C': [C],
        'gamma': [gamma]
    }

# Lista para armazenar os resultados
results = []

# Aplicar GridSearchCV para K = 2, 3, ..., 5
for i in range(1, 5):
  K = i+1
  print(f'Validação K-Fold com {K} folds em andamento...')
  grid = GridSearchCV(estimator = SVC(), param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)
  y_pred = grid.predict(X_test)
  results.append(accuracy_score(y_test, y_pred))
  print('Concluida.')

In [ ]:
# Exibindo todas as acurácias
for i in range(1, 5):
  print(f'{i+1}-Fold, acurácia:', results[i-1])
# Exibindo o melhor K e sua acurácia
print(f'Melhor K = {results.index(max(results)) + 2} com acurácia = {max(results)}')

In [ ]:
# AVALIAR 10x O MELHOR K DA VALIDAÇÃO K-FOLD
K = results.index(max(results)) + 2

classes = [0, 1, 2]
nome_classes = ['baixo risco', 'médio risco', 'alto risco']
n_classes = len(nome_classes)

cf_matrix_history = list()
accuracy_history = list()
precision_history = list()
precision_macro_history = list()
recall_history = list()
recall_macro_history = list()
f1_history = list()
f1_weighted_history = list()
f1_macro_history = list()
auc_history = list()

count = 0
for i in (0, 1, 2, 5, 10, 12, 123, 1234, 12345, 42):
  count += 1

  # Redefinindo os conjuntos de treino e teste
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''
  _, X_train, X_test, y_train, y_test = func_data_3(random_state = i, output = False) # !!! defina novamente o dataset a ser utilizado !!!
  ''' === DEFINIÇÃO DO DATASET A SER UTILIZADO ==='''

  # Treinando o modelo
  grid = GridSearchCV(estimator = SVC(), param_grid = param_grid, scoring = 'accuracy', cv = K, verbose = 0)
  grid_result = grid.fit(X_train, y_train)

  # Salvando os valores de acurácia geral e precision, recall e f1 de cada classe
  y_pred = grid.predict(X_test)

  # salvando a matriz de confusão
  cm = confusion_matrix(y_test, y_pred)
  cm = pd.DataFrame(cm, index = nome_classes, columns = nome_classes)
  cm = cm.div(cm.sum(axis = 1), axis = 0) # normalizando a matriz de confusão
  cf_matrix_history.append(cm)

  # salvando a acurácia
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_history.append(accuracy)

  # salvando a precisão de cada classe e a macro
  precision = precision_score(y_test, y_pred, average = None, zero_division = 0)
  precision_history.append(precision)
  precision = precision_score(y_test, y_pred, average = 'macro', zero_division = 0)
  precision_macro_history.append(precision)

  # salvando a recall de cada classe e a macro
  recall = recall_score(y_test, y_pred, average = None, zero_division = 0)
  recall_history.append(recall)
  recall = recall_score(y_test, y_pred, average = 'macro', zero_division = 0)
  recall_macro_history.append(recall)

  # salvando o f1-score de cada classe, o macro e o weighted
  f1 = f1_score(y_test, y_pred, average = None, zero_division = 0)
  f1_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'weighted', zero_division = 0)
  f1_weighted_history.append(f1)
  f1 = f1_score(y_test, y_pred, average = 'macro', zero_division = 0)
  f1_macro_history.append(f1)

  # salvando a média auc macro
  y_score = final_model.predict_proba(X_test) # Previsões
  y_test_bin = preprocessing.label_binarize(y_test, classes = classes) # Binarizendo as classes verdadeiras
  auc_history.append(roc_auc_score(y_test_bin, y_score, average = 'macro'))

In [ ]:
# EXIBIÇÃO DAS MÉTRICAS
print('== Resultados Gerais ==')
print(f'Acurácia: {np.array(accuracy_history).mean():.2f} ± {np.array(accuracy_history).std():.2f}')
print(f'F1 Weighted: {np.array(f1_weighted_history).mean():.2f} ± {np.array(f1_weighted_history).std():.2f}')
print(f'F1 Macro: {np.array(f1_macro_history).mean():.2f} ± {np.array(f1_macro_history).std():.2f}')
print(f'Precision: {np.array(precision_macro_history).mean():.2f} ± {np.array(precision_macro_history).std():.2f}')
print(f'Recall: {np.array(recall_macro_history).mean():.2f} ± {np.array(recall_macro_history).std():.2f}')
print()
print('== Resultados por Classe ==')
for i in range(0, n_classes):
  print(f'Precision da classe {nome_classes[i]}: {np.array(precision_history).mean(axis=0)[i]:.2f} ± {np.array(precision_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'Recall da classe {nome_classes[i]}: {np.array(recall_history).mean(axis=0)[i]:.2f} ± {np.array(recall_history).std(axis=0)[i]:.2f}')
print()
for i in range(0, n_classes):
  print(f'F1 da classe {nome_classes[i]}: {np.array(f1_history).mean(axis=0)[i]:.2f} ± {np.array(f1_history).std(axis=0)[i]:.2f}')
print()

# EXIBIÇÃO DA MATRIZ DE CONFUSÃO
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title('Matriz de Confusão')
# Criação da matriz de confusão a partir da média obtida
cf_matrix = np.array(cf_matrix_history).mean(axis=0)
cm = ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = nome_classes)
# Plotando a matriz de confusão
ax = cm.plot(ax=ax, cmap = 'Blues')

# EXIBIÇÃO DA CURVA ROC E SUA AUC
# Calculando fpr, tpr e auc para cada classe
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
  fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])

# Plotagem
plt.figure(figsize=(8, 6))
for i in range(0, n_classes):
  plt.plot(fpr[i], tpr[i], color = paleta[i], label = f'Curva ROC de {nome_classes[i]} (AUC = {roc_auc[i]:.2f})', lw = 1)

plt.plot([0, 1], [0, 1], 'k--', lw = 1)
plt.xlabel('Taxa de Falsos Positivos')
plt.xlim([-0.02, 1.0])
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.ylim([0.0, 1.02])
plt.title('Curvas ROC por classe')
plt.legend(loc="lower right")
plt.show()
print(f'AUC geral: {np.array(auc_history).mean():.2f} ± {np.array(auc_history).std():.2f}')